In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [2]:
mchirp = None
if mchirp and mchirp > 22:
    print("works")

In [3]:
from trigger_utils.trigger_utils import query_kowalski_ztf_queue
import yaml
import requests
import json
from astropy.time import Time

with open("../config/Credentials.yaml", "r") as file:
    credentials = yaml.safe_load(file)
fritztoken = credentials["fritz_token"]
ztf_allocation = credentials["allocation"]
kowalskiusername = credentials["kowalski_username"]
kowalskipassword = credentials["kowalski_password"]

# check ZTF queue

In [4]:
# print the current queue

current_ztf_queue = query_kowalski_ztf_queue(None, fritztoken, ztf_allocation)
current_ztf_queue

['Partnership_Plane_2025-07-29',
 'Partnership_Plane_2025-07-31',
 'Partnership_Plane_2025-08-02',
 'Partnership_Plane_2025-08-04',
 'Twilight_2025-07-24_e',
 'Twilight_2025-07-24_m',
 'Twilight_2025-07-25_e',
 'Twilight_2025-07-25_m',
 'Twilight_2025-07-26_e',
 'Twilight_2025-07-26_m',
 'Twilight_2025-07-27_e',
 'Twilight_2025-07-27_m',
 'Twilight_2025-07-28_e',
 'Twilight_2025-07-28_m',
 'Twilight_2025-07-29_e',
 'Twilight_2025-07-29_m',
 'Twilight_2025-07-30_e',
 'Twilight_2025-07-30_m',
 'Twilight_2025-07-31_e',
 'Twilight_2025-07-31_m',
 'Twilight_2025-08-01_e',
 'Twilight_2025-08-01_m',
 'Twilight_2025-08-02_e',
 'Twilight_2025-08-02_m',
 'Twilight_2025-08-03_e',
 'Twilight_2025-08-03_m',
 'Twilight_2025-08-04_e',
 'Twilight_2025-08-04_m',
 'default',
 'fallback',
 'missed_obs']

In [4]:
# get plan stats

from trigger_utils.trigger_utils import get_plan_stats

# get a specific plan
gcnevent_id = 13342
queuename = "S250319bu_BBHBot_2025-04-03 01:18:21.260"
stats = get_plan_stats(gcnevent_id, queuename, fritztoken, mode="")

Already submitted to queue
Total time: 2940, probability: 0.7988238394848364


# check ZTF executed observations

In [6]:
import importlib
import sys

importlib.reload(sys.modules["trigger_utils.trigger_utils"])

<module 'trigger_utils.trigger_utils' from '/home/kira/mypy/repos/BBHBot/trigger_utils/trigger_utils.py'>

In [13]:
from trigger_utils.trigger_utils import SkymapCoverage

# example skymap coverage calculation
dateobs = "2025-03-19T06:25:36"
jdstart = Time("2025-07-11T00:00:00").jd - 2
jdend = jdstart + 2
skymap = "bayestar.multiorder.fits,2"

coverage = SkymapCoverage(
    startdate=jdstart,
    enddate=jdend,
    localdateobs=dateobs,
    localname=skymap,
    localprob=0.9,
    fritz_token=fritztoken,
    kowalski_username=kowalskiusername,
    kowalski_password=kowalskipassword,
)

frac = coverage.get_coverage_fraction()
print(frac)

found 1313 exposures of 420 unique fields between JD 2460774 and 2460776
found 50 / 76 fields observed in time period
0.6578947368421053


In [9]:
headers = {"Authorization": f"token {fritztoken}"}
endpoint = f"https://fritz.science/api/instrument/1?localizationDateobs={dateobs}&localizationName={skymap}&localizationCumprob={0.9}"
response = requests.request("GET", endpoint, headers=headers)
if response.status_code != 200:
    raise Exception(
        f"API call to ZTF queue failed with status code {response.status_code}. "
        f"Response: {response.text}. Endpoint: {endpoint}"
    )
json_string = response.content.decode("utf-8")
json_data = json.loads(json_string)
fields = [
    x["field_id"]
    for x in json_data["data"]["fields"]
    if x["instrument_id"] == 1 and 220 < x["field_id"] < 880 and x["dec"] > -30
]
print(len(fields))

76


In [ ]:
# TODO: function to print table of observations overlapping skymap given a time period
# TODO: plot observations on GW skymap localization (color bar based on number of observations in each field)

# crossmatch analysis

In [ ]:
from astropy.io import fits

In [ ]:
def get_cones(path, cumprob):  # path or file-like object
    # max_order = None
    with fits.open(path) as hdul:
        hdul[1].columns
        data = hdul[1].data
    return data

In [ ]:
# https://github.com/dmitryduev/penquins/pull/21/files
# https://github.com/Theodlz/penquins/blob/abb7e9c87f8df3061315b0ce59174536e19db529/penquins/penquins.py

# other

In [1]:
import pandas as pd

In [14]:
df = pd.read_csv("../data/trigger_data/triggered_events.csv")
superevent_id = "S250319bu"  #'S250712cd'
triggered = (
    superevent_id in df["superevent_id"].values
    and df.loc[df["superevent_id"] == superevent_id, "pending_observation"].any()
)
if triggered:
    pending = df[df["superevent_id"] == superevent_id]["pending_observation"].values[0]
    trigger_plan_id = int(pending.strip("()").split(",")[0])
else:
    trigger_plan_id = None